## NLPoetry

### Generating poems using word embeddings from specified poets
#### Inspired by [Benjamín Durán's AI poetry](https://towardsdatascience.com/creating-a-poems-generator-using-word-embeddings-bcc43248de4f)


In [130]:
import os
import pandas as pd
import numpy as np
import re
import spacy
import urllib.request
from bs4 import BeautifulSoup as bs
import pandas as pd
import random

writer_list = {
    "Maya Angelou" : "maya-angelou-poems",
    "Langston Hughes" : "langston-hughes-poems",
    "Pablo Neruda" : "pablo-neruda-poems",
    "William Wordsworth" : "william-wordsworth-poems",
    "William Shakespeare" : "william-shakespeare-poems",
    "Charles Bukowski" : "charles-bukowski-poems",
    "Robert Burns" : "robert-burns-poems",
    "Rabindranath Tagore" : "rabindranath-tagore-poems",
    "Rudyard Kipling" : "rudyard-kipling-poems",
    "Robert Frost" : "robert-frost-poems",
    "Edgar Allan Poe" : "edgar-allan-poe-poems",
    "Sylvia Plath" : "sylvia-plath-poems",
    "Walt Whitman" : "walt-whitman-poems",
    "William Blake" : "william-blake-poems",
    "Dylan Thomas" : "dylan-thomas-poems",
    "John Keats" : "john-keats-poems",
    "Wilfred Owen" : "wilfred-owen-poems",
    "Roald Dahl" : "roald-dahl-poems"
}

class AppURLopener(urllib.request.FancyURLopener): 
    version = "Mozilla/5.0" 
opener = AppURLopener()
writer = random.choice(list(writer_list.values()))
print(writer)
data = opener.open('https://mypoeticside.com/poets/' + writer).read().decode()

poem_list = bs(data, 'html.parser').find(class_="list-poems")
results = ["https:"+link.get('href') for link in poem_list.findAll('a')]

titles = []
content = []

for page in results:
     data = opener.open(page).read().decode()
     soup = bs(data, 'html.parser')
    
     poem_title = soup.find(class_='title-poem')
     poem_content = soup.find(class_='poem-entry')
    
     titles.append(poem_title.getText())
     print(poem_title.getText())
    
     content.append(poem_content.find('p').getText())
     
 #saves to a .csv file all the poems   
poems = pd.DataFrame({'title' : titles, 'text' : content})
csv_name = (writer + '.csv').replace("-", "_")
poems.to_csv(csv_name)

def poem_parsing(file, split = r"\n"):
    docs = pd.read_csv(os.getcwd() + "/" + file)
    num_docs = docs.shape[0]
    sentences = pd.DataFrame(columns=['doc_id', 'sentence'])
    
    for i in range(num_docs):
        poem = docs.text[i].lower()
        
        change_chars = {
            '?«' :  '«', 
            '(' :  '', 
            ')' : '', 
            ':' : ',', 
            '.' : ',', 
            ',,,' : ',', 
            '"' : ''
        }
        
        for x, y in change_chars.items():
            poem = poem.replace(x,y)
        
        s = re.split(split, poem)
        doc_id = [i] * len(s)
        sen_dic = {'doc_id' : doc_id, 'sentence' : s}
        
        sentences = sentences.append(pd.DataFrame(sen_dic))
    print(sentences)
        
    sentences = sentences[sentences.sentence != '']
    sentences.reset_index(drop=True, inplace=True)  
    sentences.to_csv("sentences_" + file)
    
poem_parsing(csv_name)


/Users/jordanramos/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods


robert-burns-poems
A Bard's Epitaph
A Bottle And Friend
A Dedication
A Dream
A Fiddler In The North
A Man's a Man for A' That
A Poets Welcome to His Love-Begotten Daughter
A Red, Red Rose
A Winter Night
Address To A Haggis
Address to the Devil
Address To The Tooth-Ache
Address to the Unco Guid
Ae Fond Kiss
Afton Water
Again Rejoicing Nature Sees
Ah, Woe Is Me, My Mother Dear
Anna, thy Charms
Auld Lang Syne
Bonie Lesley
Bonie Peggy Alison
Ca' the Yowes to the Knowes
Comin Thro' The Rye
Craigieburn Wood
Despondency -- An Ode
Duncan Gray
Epistle to a Young Friend
Epistle to J. Lapraik (excerpt)
Epitaph on Holy Willie
Fareweel To A'Our Scottish Fame
First Six Verses Of The Ninetieth Psalm Versified, The
For A' That
From Lines to William Simson
Go Fetch to Me a Pint
Green Grow The Rashes
Halloween
Handsome Nell
Here's A Health To Them That's Awa
Here's To Thy Health
Highland Mary
Holy Willie's Prayer
I Dream'd I Lay
In The Character Of A Ruined Farmer
It Was A' For Our Rightfu' King
John An

In [131]:
def poem_generator(file, word, num_sen=4):
    spacy_nlp = spacy.load("en")
    subject = spacy_nlp(word)
    sentences = pd.read_csv(os.getcwd() +'/'+ file).fillna("")
    poem_id = int()
    poem_lst = []
    
    for i in range(num_sen):
        rand = np.random.randint(0, sentences.shape[0], size = 30)
        docs = spacy_nlp.pipe(list(sentences.sentence.iloc[rand]))
        
        similarities = []
        for s in docs:
            similarities.append(spacy_nlp(word).similarity(s))
            
        s_dict = {
            'similarity' : similarities,
            'doc_id' : sentences.doc_id.iloc[rand]
        }
        
        df_sim = pd.DataFrame(s_dict, index=rand)
        df_sim = df_sim[df_sim.doc_id != poem_id]
        df_sim.sort_values(by='similarity', inplace=True, ascending=False)
        
        s = sentences.sentence[df_sim.index[0]]
        
        change_chars = {
            '\n' :  '', 
            '\r' :  '', 
        }
        
        for x, y in change_chars.items():
            s = s.replace(x, y)
        
        poem_lst.append(s)
        poem_id = df_sim.doc_id.iloc[0]
        subject = spacy_nlp(s)
    
    poem = ("\n".join(poem_lst))
    return poem

In [132]:
def poem_format(text):
    return (text[:1].upper() + text[1:])[:-1] + '.'

In [133]:
sentences_csv = "sentences_" + csv_name

poem = poem_generator(file = sentences_csv, word='valor')
poem = poem_format(poem)
print(poem)

/Users/jordanramos/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  


My fate will scarce bestow,
they've lost some gallant gentlemen,
dearest tie of young connections,
but with humility and aw.
